# Archive

## Backtesting module Example

In [93]:
class SmaCross(Strategy):
    def init(self):
        price = self.data.Close
        self.ma1 = self.I(SMA, price, 10)
        self.ma2 = self.I(SMA, price, 20)

    def next(self):
        if crossover(self.ma1, self.ma2):
            self.buy()
        elif crossover(self.ma2, self.ma1):
            self.sell()




In [4]:
data = yf.download('SPY', start='2018-01-01')

[*********************100%***********************]  1 of 1 completed


In [5]:
# bt = Backtest(data, SmaCross, commission=.002,
#               exclusive_orders=True)
# stats = bt.run()
# bt.plot()

In [6]:
# stats

## yFinance Wrapper

In [ ]:
from modules import utils

start_year = 2022
start_month = 9
start_day = 2

# end_year = 2022
# end_month = 9
# end_day = 27


# Valid intervals: [1m, 2m, 5m, 15m, 30m, 60m, 90m, 1h, 1d, 5d, 1wk, 1mo, 3mo]
interval = '1h'

data = utils.pull_stock(
    symbol='SPY',
    interval=interval,
    start_year=start_year,
    start_month=start_month,
    start_day=start_day,
    end_year=end_year,
    end_month=end_month,
    end_day=end_day,
)

data

# First bullish reversal

In [ ]:
def bullish_reversal(df_data):
    '''
    Gets a Pandas DataFrame with OHLC data and returns whether the last few candlesticks indicate a bullish reversal (down to up).
    
    Input:
        df_data (pd.DataFrame) - OHLC data.
    
    Returns:
        bull (boolean) - A boolean indicator, telling whether there has just now been a bullish reversal.
    '''
    
    # we go in reverse chronological order, starting with the most recent candlestick
    df_data = df_data.iloc[::-1]

    
    # first we check for confirmation - is the most recent candlestick bullish?
    last_candle = df_data.iloc[0]
    confirmation = last_candle['Close'] - last_candle['Open'] > 0
    
    if not confirmation: return False
    
    # afterwards, we allow up to two more consecutive bulls before seeing a bearish one
    max_bull = 2
    
    # we assume there are over <max_bull> consecutive bulls before the confirmation, until proven otherwise
    too_many_bulls = True  
    
    # goes from df_data[-2] to df_data[-max_bull-1] and checks if those candles are bullish
    for i in range(2, max_bull + 2):  
        cur_candle = df_data.iloc[i]
        is_bull = cur_candle['Close'] - cur_candle['Open'] > 0
        
        # it's a bearish candlestick!
        if not is_bull:  
            too_many_bulls = False  # the assumption is broken - we are proven otherwise
            bear_idx = i
            break  
    
    # in this case, we consider this a bullish trend, not a reversal
    if too_many_bulls: return False  
    
    # if we reched here, we have a confirmation, not too many bulls and now a bear
    bear_candle = df_data.iloc[bear_idx]
    
    # now we check if this candle is a pattern indicaing a bullish reversal
    # hammer
    
    bull_reversal = True